##### After reviewing the data available from the ByBit API and only being able to access the previous 200 instances for items such as open interest, as well as not having appropriate data for buy vs. sell pressure (79 missing values out of 200), I have decided not to pursue a new predictor model using this data and instead to attempt to initialize a trading bot from my exisiting predictor.

Want to attempt to automate a ByBit trading bot that will automatically enter into Bitcoin trading positions based on a predictive model that will also be created using the available data from the ByBit API endpoints.

I will be using similar information to a previous project on Bitcoin Price Direction Movement (https://github.com/shmaze/LHL-Final-Project). However will be adding in more features based on available endpoints.

One features we hope to add is open interest, will be exploring more data as well.

In [1]:
#!pip install pybit

In [2]:
# this should cover all necessafry packages 

import pandas as pd
import numpy as np
from pybit import HTTP
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline, FeatureUnion
from xgboost import XGBClassifier
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score, confusion_matrix, ConfusionMatrixDisplay, r2_score, mean_squared_error, mean_absolute_error, matthews_corrcoef
import joblib

In [3]:
import time
import math

In [4]:
now = time.time()

math.trunc(time.time())

1641428838

In [5]:

# now we will test out the PyBit library to see what kind of data we can get from their API endpoints that we can use as input features for our predictor model


session = HTTP("https://api-testnet.bybit.com")
data = session.query_kline(symbol='BTCUSD',
                           interval='D',
                           from_time=math.trunc(time.time()) - 3600 * 24 * 200
                          )

data


# this returns hourly data starting from 24 hours prior to the time it was run

{'ret_code': 0,
 'ret_msg': 'OK',
 'ext_code': '',
 'ext_info': '',
 'result': [{'symbol': 'BTCUSD',
   'interval': 'D',
   'open_time': 1624233600,
   'open': '35634.5',
   'high': '35732.5',
   'low': '31300',
   'close': '31662.5',
   'volume': '2043327664',
   'turnover': '61945.35026791'},
  {'symbol': 'BTCUSD',
   'interval': 'D',
   'open_time': 1624320000,
   'open': '31662.5',
   'high': '33303',
   'low': '28811',
   'close': '32508.5',
   'volume': '1886255415',
   'turnover': '59812.47288134'},
  {'symbol': 'BTCUSD',
   'interval': 'D',
   'open_time': 1624406400,
   'open': '32508.5',
   'high': '34863.5',
   'low': '31769.5',
   'close': '33664.5',
   'volume': '1667823154',
   'turnover': '49410.19040348'},
  {'symbol': 'BTCUSD',
   'interval': 'D',
   'open_time': 1624492800,
   'open': '33664.5',
   'high': '35300.5',
   'low': '32304',
   'close': '34684',
   'volume': '2029650624',
   'turnover': '60254.95240603'},
  {'symbol': 'BTCUSD',
   'interval': 'D',
   'open_

In [6]:
data_new = data['result']
data_new

[{'symbol': 'BTCUSD',
  'interval': 'D',
  'open_time': 1624233600,
  'open': '35634.5',
  'high': '35732.5',
  'low': '31300',
  'close': '31662.5',
  'volume': '2043327664',
  'turnover': '61945.35026791'},
 {'symbol': 'BTCUSD',
  'interval': 'D',
  'open_time': 1624320000,
  'open': '31662.5',
  'high': '33303',
  'low': '28811',
  'close': '32508.5',
  'volume': '1886255415',
  'turnover': '59812.47288134'},
 {'symbol': 'BTCUSD',
  'interval': 'D',
  'open_time': 1624406400,
  'open': '32508.5',
  'high': '34863.5',
  'low': '31769.5',
  'close': '33664.5',
  'volume': '1667823154',
  'turnover': '49410.19040348'},
 {'symbol': 'BTCUSD',
  'interval': 'D',
  'open_time': 1624492800,
  'open': '33664.5',
  'high': '35300.5',
  'low': '32304',
  'close': '34684',
  'volume': '2029650624',
  'turnover': '60254.95240603'},
 {'symbol': 'BTCUSD',
  'interval': 'D',
  'open_time': 1624579200,
  'open': '34684',
  'high': '35518',
  'low': '31292.5',
  'close': '31600.5',
  'volume': '21291

In [7]:
data_dict = {}
for entry in range(len(data_new)):
    features = []
    time = data_new[entry]['open_time']
    open_val = data_new[entry]['open']
    close_val = data_new[entry]['close']
    high_val = data_new[entry]['high']
    low_val = data_new[entry]['low']
    volume = data_new[entry]['volume']
    turnover = data_new[entry]['turnover']
    features.append(open_val)
    features.append(close_val)
    features.append(high_val)
    features.append(low_val)
    features.append(volume)
    features.append(turnover)
    
    data_dict[time] = features
    
    

In [8]:
data_dict

{1624233600: ['35634.5',
  '31662.5',
  '35732.5',
  '31300',
  '2043327664',
  '61945.35026791'],
 1624320000: ['31662.5',
  '32508.5',
  '33303',
  '28811',
  '1886255415',
  '59812.47288134'],
 1624406400: ['32508.5',
  '33664.5',
  '34863.5',
  '31769.5',
  '1667823154',
  '49410.19040348'],
 1624492800: ['33664.5',
  '34684',
  '35300.5',
  '32304',
  '2029650624',
  '60254.95240603'],
 1624579200: ['34684',
  '31600.5',
  '35518',
  '31292.5',
  '2129157577',
  '63937.47896722'],
 1624665600: ['31600.5',
  '32287.5',
  '32753.5',
  '30149.5',
  '1949772160',
  '62087.01732756'],
 1624752000: ['32287.5',
  '34686.5',
  '34742.5',
  '31994.5',
  '1893492131',
  '57292.6782675'],
 1624838400: ['34686.5',
  '34474.5',
  '35338.5',
  '33887.5',
  '1474120296',
  '42722.45794125'],
 1624924800: ['34474.5',
  '35902.5',
  '36642.5',
  '34251',
  '1673595187',
  '47234.63602609'],
 1625011200: ['35902.5',
  '35033.5',
  '36092',
  '34085.5',
  '1544434356',
  '44288.4983548'],
 162509760

In [9]:
open_int = session.open_interest(
    symbol="BTCUSD",
    period="1d",
    limit=200
)

open_int

{'ret_code': 0,
 'ret_msg': 'OK',
 'ext_code': '',
 'ext_info': '',
 'result': [{'open_interest': 261271217,
   'timestamp': 1641427200,
   'symbol': 'BTCUSD'},
  {'open_interest': 268813508, 'timestamp': 1641340800, 'symbol': 'BTCUSD'},
  {'open_interest': 275887697, 'timestamp': 1641254400, 'symbol': 'BTCUSD'},
  {'open_interest': 267986464, 'timestamp': 1641168000, 'symbol': 'BTCUSD'},
  {'open_interest': 266908910, 'timestamp': 1641081600, 'symbol': 'BTCUSD'},
  {'open_interest': 260968171, 'timestamp': 1640995200, 'symbol': 'BTCUSD'},
  {'open_interest': 269450299, 'timestamp': 1640908800, 'symbol': 'BTCUSD'},
  {'open_interest': 268083458, 'timestamp': 1640822400, 'symbol': 'BTCUSD'},
  {'open_interest': 277417809, 'timestamp': 1640736000, 'symbol': 'BTCUSD'},
  {'open_interest': 276872296, 'timestamp': 1640649600, 'symbol': 'BTCUSD'},
  {'open_interest': 274901527, 'timestamp': 1640563200, 'symbol': 'BTCUSD'},
  {'open_interest': 287203272, 'timestamp': 1640476800, 'symbol': 'BT

In [10]:
openInt = open_int['result']

In [11]:
int_dict = {}

for entry in range(len(openInt)):
    time = openInt[entry]['timestamp']
    interest = openInt[entry]['open_interest']
    
    int_dict[time] = interest

In [12]:
int_dict

{1641427200: 261271217,
 1641340800: 268813508,
 1641254400: 275887697,
 1641168000: 267986464,
 1641081600: 266908910,
 1640995200: 260968171,
 1640908800: 269450299,
 1640822400: 268083458,
 1640736000: 277417809,
 1640649600: 276872296,
 1640563200: 274901527,
 1640476800: 287203272,
 1640390400: 284248435,
 1640304000: 272621243,
 1640217600: 290800029,
 1640131200: 290344331,
 1640044800: 286433477,
 1639958400: 270319121,
 1639872000: 268195780,
 1639785600: 246669265,
 1639699200: 261821091,
 1639612800: 248561571,
 1639526400: 257280429,
 1639440000: 250631824,
 1639353600: 302369879,
 1639267200: 279361167,
 1639180800: 269061086,
 1639094400: 254944434,
 1639008000: 254480218,
 1638921600: 267155962,
 1638748800: 240873223,
 1638662400: 216672270,
 1638576000: 368429886,
 1638489600: 373492934,
 1638403200: 375353688,
 1638316800: 380375542,
 1638230400: 375580341,
 1638144000: 368979179,
 1638057600: 387594383,
 1637971200: 370665925,
 1637884800: 411801032,
 1637798400: 384

In [13]:
longShort = session.long_short_ratio(
    symbol="BTCUSD",
    period="1d",
    limit=200
)

longShort

{'ret_code': 0,
 'ret_msg': 'OK',
 'ext_code': '',
 'ext_info': '',
 'result': [{'symbol': 'BTCUSD',
   'buy_ratio': 0.5285,
   'sell_ratio': 0.4715,
   'timestamp': 1641427200},
  {'symbol': 'BTCUSD',
   'buy_ratio': 0.5297,
   'sell_ratio': 0.4703,
   'timestamp': 1641340800},
  {'symbol': 'BTCUSD',
   'buy_ratio': 0.5297,
   'sell_ratio': 0.4703,
   'timestamp': 1641254400},
  {'symbol': 'BTCUSD',
   'buy_ratio': 0.5281,
   'sell_ratio': 0.4719,
   'timestamp': 1641168000},
  {'symbol': 'BTCUSD',
   'buy_ratio': 0.5284,
   'sell_ratio': 0.4716,
   'timestamp': 1641081600},
  {'symbol': 'BTCUSD',
   'buy_ratio': 0.5287,
   'sell_ratio': 0.4713,
   'timestamp': 1640995200},
  {'symbol': 'BTCUSD',
   'buy_ratio': 0.5285,
   'sell_ratio': 0.4715,
   'timestamp': 1640908800},
  {'symbol': 'BTCUSD',
   'buy_ratio': 0.5281,
   'sell_ratio': 0.4719,
   'timestamp': 1640822400},
  {'symbol': 'BTCUSD',
   'buy_ratio': 0.5281,
   'sell_ratio': 0.4719,
   'timestamp': 1640736000},
  {'symbol': 

In [14]:
longshort = longShort['result']

In [15]:
longAndShort = {}
for entry in range(len(longshort)):
    ratios = []
    buy = longshort[entry]['buy_ratio']
    sell = longshort[entry]['sell_ratio']
    time = longshort[entry]['timestamp']
    ratios.append(buy)
    ratios.append(sell)
    longAndShort[time] = ratios
    

In [16]:
longAndShort

{1641427200: [0.5285, 0.4715],
 1641340800: [0.5297, 0.4703],
 1641254400: [0.5297, 0.4703],
 1641168000: [0.5281, 0.4719],
 1641081600: [0.5284, 0.4716],
 1640995200: [0.5287, 0.4713],
 1640908800: [0.5285, 0.4715],
 1640822400: [0.5281, 0.4719],
 1640736000: [0.5281, 0.4719],
 1640649600: [0.527, 0.473],
 1640563200: [0.5267, 0.4733],
 1640476800: [0.5265, 0.4735],
 1640390400: [0.5269, 0.4731],
 1640304000: [0.5265, 0.4735],
 1640217600: [0.5256, 0.4744],
 1640131200: [0.5261, 0.4739],
 1640044800: [0.5251, 0.4749],
 1639958400: [0.526, 0.474],
 1639872000: [0.5258, 0.4742],
 1639785600: [0.5263, 0.4737],
 1639699200: [0.5256, 0.4744],
 1639612800: [0.5264, 0.4736],
 1639526400: [0.5263, 0.4737],
 1639440000: [0.5264, 0.4736],
 1639353600: [0.5252, 0.4748],
 1639267200: [0.5252, 0.4748],
 1639180800: [0.5263, 0.4737],
 1639094400: [0.5266, 0.4734],
 1639008000: [0.5265, 0.4735],
 1638921600: [0.5262, 0.4738],
 1638748800: [0.5274, 0.4726],
 1638662400: [0.5279, 0.4721],
 1638576000:

In [23]:
print(longAndShort.get(1624060700))

None


In [24]:
master_list = []
for key in data_dict.keys():
    daily_values = []
    time = key
    daily_values.append(time)
    data_list = data_dict.get(key)
    for item in data_list:
        daily_values.append(item)
    open_int = int_dict.get(key)
    daily_values.append(open_int)
    buy_sell = longAndShort.get(key)
    try:
        for value in buy_sell:
        
            daily_values.append(value)
    except TypeError:
        pass
            
    master_list.append(daily_values)
    

In [26]:
len(master_list)


200

In [27]:
df = pd.DataFrame(master_list, columns=['Time', 'Open', 'Close', 'High', 'Low', 'Volume', 'Turnover', 'Open_Interest',
                                       'Buy', 'Sell'])

In [28]:
df.dtypes

Time               int64
Open              object
Close             object
High              object
Low               object
Volume            object
Turnover          object
Open_Interest    float64
Buy              float64
Sell             float64
dtype: object

In [29]:
df.Open = df.Open.astype('float')

In [30]:
df.dtypes

Time               int64
Open             float64
Close             object
High              object
Low               object
Volume            object
Turnover          object
Open_Interest    float64
Buy              float64
Sell             float64
dtype: object

In [31]:
df.Close = df.Close.astype('float')
df.High = df.High.astype('float')
df.Low = df.Low.astype('float')
df.Volume = df.Volume.astype('float')
df.Turnover = df.Turnover.astype('float')

In [32]:
df.dtypes

Time               int64
Open             float64
Close            float64
High             float64
Low              float64
Volume           float64
Turnover         float64
Open_Interest    float64
Buy              float64
Sell             float64
dtype: object

In [33]:
df.Time = df.Time.apply(lambda x: pd.to_datetime(x, unit='s'))

In [34]:
df.dtypes

Time             datetime64[ns]
Open                    float64
Close                   float64
High                    float64
Low                     float64
Volume                  float64
Turnover                float64
Open_Interest           float64
Buy                     float64
Sell                    float64
dtype: object

In [35]:
df.set_index('Time', inplace=True)

In [36]:
df

Open    Close     High      Low        Volume      Turnover  \
Time                                                                         
2021-06-21  35634.5  31662.5  35732.5  31300.0  2.043328e+09  61945.350268   
2021-06-22  31662.5  32508.5  33303.0  28811.0  1.886255e+09  59812.472881   
2021-06-23  32508.5  33664.5  34863.5  31769.5  1.667823e+09  49410.190403   
2021-06-24  33664.5  34684.0  35300.5  32304.0  2.029651e+09  60254.952406   
2021-06-25  34684.0  31600.5  35518.0  31292.5  2.129158e+09  63937.478967   
...             ...      ...      ...      ...           ...           ...   
2022-01-02  47737.0  47305.0  48002.0  46700.0  6.536488e+07   1382.888187   
2022-01-03  47305.0  46415.0  47544.0  45542.0  1.026239e+08   2205.101030   
2022-01-04  46415.0  45828.5  47521.0  45201.5  1.726107e+08   3716.533806   
2022-01-05  45828.5  43501.0  47014.5  42500.0  2.415808e+08   5363.204346   
2022-01-06  43501.0  43460.5  43803.0  43264.5  4.653961e+06    106.987265   

            Open_Interest     Buy    Sell  
Time                                       
2021-06-21    109637181.0  0.5428  0.4572  
2021-06-22    120385034.0  0.5423  0.4577  
2021-06-23    105991378.0  0.5372  0.4628  
2021-06-24    119167118.0  0.5382  0.4618  
2021-06-25    134265054.0     NaN     NaN  
...                   ...     ...     ...  
2022-01-02    266908910.0  0.5284  0.4716  
2022-01-03    267986464.0  0.5281  0.4719  
2022-01-04    275887697.0  0.5297  0.4703  
2022-01-05    268813508.0  0.5297  0.4703  
2022-01-06    261271217.0  0.5285  0.4715  

[200 rows x 9 columns]

In [37]:
df.isnull().sum()

Open              0
Close             0
High              0
Low               0
Volume            0
Turnover          0
Open_Interest     1
Buy              79
Sell             79
dtype: int64

In [38]:
from binance.client import Client

client = Client()


candles = client.get_klines(symbol='BTCUSDT', interval=Client.KLINE_INTERVAL_1DAY, limit=1000)


In [39]:
candles

[[1555113600000,
  '5047.00000000',
  '5099.00000000',
  '5004.00000000',
  '5045.22000000',
  '17292.45680200',
  1555199999999,
  '87375984.44461506',
  138674,
  '8692.05325900',
  '43925293.92220215',
  '0'],
 [1555200000000,
  '5047.45000000',
  '5152.99000000',
  '5000.00000000',
  '5131.30000000',
  '18281.60773900',
  1555286399999,
  '92594930.42710131',
  131700,
  '10043.91166700',
  '50893945.02767781',
  '0'],
 [1555286400000,
  '5131.28000000',
  '5167.38000000',
  '4950.00000000',
  '5024.95000000',
  '29057.19158100',
  1555372799999,
  '147687551.94147710',
  204944,
  '14170.25240200',
  '72070166.00084409',
  '0'],
 [1555372800000,
  '5024.95000000',
  '5197.72000000',
  '5003.94000000',
  '5173.72000000',
  '24242.22949300',
  1555459199999,
  '123581651.71216356',
  168550,
  '12804.63376800',
  '65290617.51599363',
  '0'],
 [1555459200000,
  '5173.72000000',
  '5230.40000000',
  '5146.80000000',
  '5202.82000000',
  '23307.53613400',
  1555545599999,
  '120972345.

In [40]:
data = pd.DataFrame(candles, columns=['Open time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close time', 'Quote asset volume', 'Number of trades', 'Taker buy base volume', 'Taker buy quote volume', 'Ignore'])


data

Open time            Open            High             Low  \
0    1555113600000   5047.00000000   5099.00000000   5004.00000000   
1    1555200000000   5047.45000000   5152.99000000   5000.00000000   
2    1555286400000   5131.28000000   5167.38000000   4950.00000000   
3    1555372800000   5024.95000000   5197.72000000   5003.94000000   
4    1555459200000   5173.72000000   5230.40000000   5146.80000000   
..             ...             ...             ...             ...   
995  1641081600000  47722.66000000  47990.00000000  46654.00000000   
996  1641168000000  47286.18000000  47570.00000000  45696.00000000   
997  1641254400000  46446.10000000  47557.54000000  45500.00000000   
998  1641340800000  45832.01000000  47070.00000000  42500.00000000   
999  1641427200000  43451.14000000  43816.00000000  42632.00000000   

              Close          Volume     Close time   Quote asset volume  \
0     5045.22000000  17292.45680200  1555199999999    87375984.44461506   
1     5131.30000000  18281.60773900  1555286399999    92594930.42710131   
2     5024.95000000  29057.19158100  1555372799999   147687551.94147710   
3     5173.72000000  24242.22949300  1555459199999   123581651.71216356   
4     5202.82000000  23307.53613400  1555545599999   120972345.51821637   
..              ...             ...            ...                  ...   
995  47286.18000000  18340.46040000  1641167999999   866610995.56335820   
996  46446.10000000  27662.07710000  1641254399999  1292203508.32565490   
997  45832.01000000  35491.41360000  1641340799999  1649170424.12761340   
998  43451.13000000  51784.11857000  1641427199999  2334288544.64544840   
999  42645.97000000  10195.10289000  1641513599999   441146679.66915540   

     Number of trades Taker buy base volume Taker buy quote volume Ignore  
0              138674         8692.05325900      43925293.92220215      0  
1              131700        10043.91166700      50893945.02767781      0  
2              204944        14170.25240200      72070166.00084409      0  
3              168550        12804.63376800      65290617.51599363      0  
4              179490        12172.53172400      63190167.89632924      0  
..                ...                   ...                    ...    ...  
995            709624         9166.46954000     433182392.28194660      0  
996            885624        13524.76045000     631879430.11428000      0  
997           1021815        17689.26808000     821725463.08080750      0  
998           1478532        23552.99460000    1063287226.35714140      0  
999            261541         4848.59691000     209792214.39769100      0  

[1000 rows x 12 columns]

In [41]:
data.drop(['Close time', 'Ignore'], axis=1, inplace=True)
data['Open time'] = data['Open time'].apply(lambda x: pd.to_datetime(x, unit='ms'))
data.set_index('Open time', inplace=True)
data.head()

Open           High            Low          Close  \
Open time                                                                
2019-04-13  5047.00000000  5099.00000000  5004.00000000  5045.22000000   
2019-04-14  5047.45000000  5152.99000000  5000.00000000  5131.30000000   
2019-04-15  5131.28000000  5167.38000000  4950.00000000  5024.95000000   
2019-04-16  5024.95000000  5197.72000000  5003.94000000  5173.72000000   
2019-04-17  5173.72000000  5230.40000000  5146.80000000  5202.82000000   

                    Volume  Quote asset volume  Number of trades  \
Open time                                                          
2019-04-13  17292.45680200   87375984.44461506            138674   
2019-04-14  18281.60773900   92594930.42710131            131700   
2019-04-15  29057.19158100  147687551.94147710            204944   
2019-04-16  24242.22949300  123581651.71216356            168550   
2019-04-17  23307.53613400  120972345.51821637            179490   

           Taker buy base volume Taker buy quote volume  
Open time                                                
2019-04-13         8692.05325900      43925293.92220215  
2019-04-14        10043.91166700      50893945.02767781  
2019-04-15        14170.25240200      72070166.00084409  
2019-04-16        12804.63376800      65290617.51599363  
2019-04-17        12172.53172400      63190167.89632924

In [42]:
df

Open    Close     High      Low        Volume      Turnover  \
Time                                                                         
2021-06-21  35634.5  31662.5  35732.5  31300.0  2.043328e+09  61945.350268   
2021-06-22  31662.5  32508.5  33303.0  28811.0  1.886255e+09  59812.472881   
2021-06-23  32508.5  33664.5  34863.5  31769.5  1.667823e+09  49410.190403   
2021-06-24  33664.5  34684.0  35300.5  32304.0  2.029651e+09  60254.952406   
2021-06-25  34684.0  31600.5  35518.0  31292.5  2.129158e+09  63937.478967   
...             ...      ...      ...      ...           ...           ...   
2022-01-02  47737.0  47305.0  48002.0  46700.0  6.536488e+07   1382.888187   
2022-01-03  47305.0  46415.0  47544.0  45542.0  1.026239e+08   2205.101030   
2022-01-04  46415.0  45828.5  47521.0  45201.5  1.726107e+08   3716.533806   
2022-01-05  45828.5  43501.0  47014.5  42500.0  2.415808e+08   5363.204346   
2022-01-06  43501.0  43460.5  43803.0  43264.5  4.653961e+06    106.987265   

            Open_Interest     Buy    Sell  
Time                                       
2021-06-21    109637181.0  0.5428  0.4572  
2021-06-22    120385034.0  0.5423  0.4577  
2021-06-23    105991378.0  0.5372  0.4628  
2021-06-24    119167118.0  0.5382  0.4618  
2021-06-25    134265054.0     NaN     NaN  
...                   ...     ...     ...  
2022-01-02    266908910.0  0.5284  0.4716  
2022-01-03    267986464.0  0.5281  0.4719  
2022-01-04    275887697.0  0.5297  0.4703  
2022-01-05    268813508.0  0.5297  0.4703  
2022-01-06    261271217.0  0.5285  0.4715  

[200 rows x 9 columns]

In [43]:
data.tail()

Open            High             Low           Close  \
Open time                                                                    
2022-01-02  47722.66000000  47990.00000000  46654.00000000  47286.18000000   
2022-01-03  47286.18000000  47570.00000000  45696.00000000  46446.10000000   
2022-01-04  46446.10000000  47557.54000000  45500.00000000  45832.01000000   
2022-01-05  45832.01000000  47070.00000000  42500.00000000  43451.13000000   
2022-01-06  43451.14000000  43816.00000000  42632.00000000  42645.97000000   

                    Volume   Quote asset volume  Number of trades  \
Open time                                                           
2022-01-02  18340.46040000   866610995.56335820            709624   
2022-01-03  27662.07710000  1292203508.32565490            885624   
2022-01-04  35491.41360000  1649170424.12761340           1021815   
2022-01-05  51784.11857000  2334288544.64544840           1478532   
2022-01-06  10195.10289000   441146679.66915540            261541   

           Taker buy base volume Taker buy quote volume  
Open time                                                
2022-01-02         9166.46954000     433182392.28194660  
2022-01-03        13524.76045000     631879430.11428000  
2022-01-04        17689.26808000     821725463.08080750  
2022-01-05        23552.99460000    1063287226.35714140  
2022-01-06         4848.59691000     209792214.39769100